## <b><font color='darkblue'>Preface</font></b>
Here we are going to demo in using `langfun` to write unit test case.

### <b><font color='darkgreen'>Import packages</font></b>
Let's import necessary packages:

In [57]:
!pip freeze | grep -P "(langfun)"

langfun==0.1.1


In [58]:
import pathlib
import os
import langfun as lf
import pyglove as pg


Path = pathlib.Path
print("*" * len(os.environ.get('GOOGLE_API_KEY', '')))

***************************************


In [59]:
from IPython.display import display, Markdown, Latex

def show_source_code(src_path: str):
    source_code = !cat $src_path
    display(Markdown(f"""
```python
{'\n'.join(source_code)}
```"""))

### <b><font color='darkgreen'>PyGlove: Manipulating Python Programs</font></b>
<font size='3ptx'><b>[PyGlove](https://github.com/google/pyglove) is a general-purpose library for Python object manipulation</b>. It introduces symbolic object-oriented programming to Python, allowing direct manipulation of objects that makes meta-programs much easier to write. It has been used to handle complex machine learning scenarios, such as AutoML, as well as facilitating daily programming tasks with extra flexibility.</font>

### <b><font color='darkgreen'>Langfun: PyGlove-powered Python library that makes working with language models more fun</font></b>
<font size='3ptx'><b>[Langfun](https://github.com/google/langfun) is a PyGlove powered library that aims to make language models (LM) fun to work with</b>. Its central principle is to enable seamless integration between natural language and programming by treating language as functions. Through the introduction of Object-Oriented Prompting, Langfun empowers users to prompt LLMs using objects and types, offering enhanced control and simplifying agent development.</font>

## <b><font color='darkblue'>Natural Language -> Unit test case</font></b>
To let LLM to write unit test case for us, we need to initialize the LLM and provide the context.

### <b><font color='darkgreen'>Initialization</font></b>
Let's initialize our LLM and environment:

In [60]:
[model for model in dir(lf.llms) if 'Gemini' in model]

['GeminiPro',
 'GeminiPro1_5',
 'GeminiProVision',
 'VertexAIGeminiFlash1_5',
 'VertexAIGeminiFlash1_5_0514',
 'VertexAIGeminiPro1',
 'VertexAIGeminiPro1Vision',
 'VertexAIGeminiPro1_5',
 'VertexAIGeminiPro1_5_0409',
 'VertexAIGeminiPro1_5_0514']

In [61]:
model = lf.llms.GeminiPro1_5()

In [28]:
class TestModule(pg.Object):
    """Symbolic class to hold the target module for generating unit test cases.
    
    Attributes:
      project_root_path: Path to the project root.
      module_package: Package path of the module.
      module_name: Name of the module.
      module_content: Source code of the module.
    """
    project_root_path: str
    module_package: str
    module_name: str
    module_content: str


class UnitTestCases(pg.Object):
    """Symbolic class to hold generated unit test cases.

    Attributes:
      project_root_path: Path to the project root.
      module_package: Package path of the target module.
      module_name: Name of the target module.
        test_case_content: Generated unit test case source code.
        
      project_root_path: Project root path.
      module_package: Module package path.
      module_name: Module name.
      test_case_content: Content of generated unit test cases.
      test_module_name: Testing module name to hold unit test cases.
      unit_test_root_path: Root path to hold modules of unit test cases.
      unit_test_case_module_dir_path: Directory of testing module to hold unit test cases.
      unit_test_case_module_path: Path of testing module to hold unit test cases.
      unit_test_case_module_package: Package of unit test case module.
    """
    project_root_path: str
    module_package: str
    module_name: str
    test_case_content: str

    @property
    def test_module_name(self) -> str:
        return f'test_{self.module_name}.py'
    
    @property
    def unit_test_root_path(self) -> str:
        return str(
            Path(self.project_root_path) / Path('tests/unit'))

    @property
    def unit_test_case_module_dir_path(self) -> str:
        return str(
            Path('tests/unit') / Path(self.module_package.replace('.', '/')))
    
    @property
    def unit_test_case_module_path(self) -> str:
        return str(
            Path(self.unit_test_root_path) / Path(self.module_package.replace('.', '/') / Path(self.test_module_name)))

    @property
    def unit_test_case_module_package(self) -> str:
        module_name = self.test_module_name.split(".")[0]
    
        # Find the fully qualified module path
        return f"{self.unit_test_case_module_dir_path}.{module_name}".replace('/', '.')
    
    def output(self):
        os.makedirs(os.path.dirname(self.unit_test_case_module_path), exist_ok=True)

        with open(self.unit_test_case_module_path, 'w') as fw:
            fw.write(self.test_case_content)

### <b><font color='darkgreen'>Prepare Testing Module</font></b>
Let's prepare the testing module for generation of unit test cases.

In [7]:
import my_math

help(my_math.add)

Help on function add in module my_math:

add(a: int, b: int) -> int
    Sums up the input `a` and `b`.



In [8]:
my_math.add(1, 2)

3

Prepare `TestModule` as object to generate unit test cases of it:

In [9]:
module_package = 'utils'
project_root_path = os.getcwd()
module_name = 'my_math'
my_math_module_content = open(
    Path(project_root_path) / module_package.replace('.', '/') / f'{module_name}.py', 'r').read()

test_module = TestModule(
    project_root_path=project_root_path,
    module_package = module_package,
    module_name=module_name,
    module_content=my_math_module_content,
)

In [10]:
test_module

TestModule(project_root_path='/home/john/Gitrepos/ml_articles/others/Langfun_with_unittest', module_package='utils', module_name='my_math', module_content='def add(a: int, b: int) -> int:\n  """Sums up the input `a` and `b`.\n\n  Args:\n    a: First value to add\n    b: Second value to add\n\n  Returns:\n    Return value of `a + b`.\n  """\n  return a + b\n')

### <b><font color='darkgreen'>Create Unit Test Cases</font></b>
Let's try to create unit test cases:

In [11]:
help(lf.query)

Help on function query in module langfun.core.structured.prompting:

query(prompt: Union[str, pyglove.core.symbolic.base.Symbolic], schema: Union[langfun.core.structured.schema.Schema, Type[Any], list[Type[Any]], dict[str, Any], NoneType] = None, default: Any = (MISSING_VALUE,), *, lm: langfun.core.language_model.LanguageModel | None = None, examples: list[langfun.core.structured.mapping.MappingExample] | None = None, cache_seed: int | None = 0, response_postprocess: Optional[Callable[[str], str]] = None, autofix: int = 0, autofix_lm: langfun.core.language_model.LanguageModel | None = None, protocol: Literal['json', 'python'] = 'python', returns_message: bool = False, skip_lm: bool = False, **kwargs) -> Any
    Parse a natural langugage message based on schema.

    Examples:

      ```
      class FlightDuration:
        hours: int
        minutes: int

      class Flight(pg.Object):
        airline: str
        flight_number: str
        departure_airport_code: str
        arrival_ai

In [62]:
unit_test_case_result = lf.query(
    prompt='Please create unit test cases for {{module}}',  # Prompt to request the generation of test cases.
    schema=UnitTestCases,                                   # A type annotation as the schema for output object.
    lm=model,                                               # The language model to use.
    module=test_module,                                     # The value of placeholder '{{module}}'
)

In [63]:
unit_test_case_result

UnitTestCases(project_root_path='/home/john/Gitrepos/ml_articles/others/Langfun_with_unittest', module_package='utils', module_name='my_math', test_case_content='import unittest\nfrom utils.my_math import add\n\n\nclass TestAdd(unittest.TestCase):\n    def test_positive_integers(self):\n        self.assertEqual(add(1, 2), 3)\n\n    def test_negative_integers(self):\n        self.assertEqual(add(-1, -2), -3)\n\n    def test_zero_sum(self):\n        self.assertEqual(add(0, 0), 0)\n\n    def test_positive_and_negative(self):\n        self.assertEqual(add(1, -2), -1)\n')

In [31]:
# Path of testing module holding the generated unit test cases.
unit_test_case_result.unit_test_case_module_path

'/home/john/Gitrepos/ml_articles/others/Langfun_with_unittest/tests/unit/utils/test_my_math.py'

In [32]:
unit_test_case_result.unit_test_case_module_package

'tests.unit.utils.test_my_math'

### <b><font color='darkgreen'>Execute Generated Unit Test Cases</font></b>
Below will demonstrate on how to execute the generated unit test cases:

#### <b>Output unit test case content into file</b>

In [33]:
# Output the unit test cases into file:
os.makedirs(os.path.dirname(unit_test_case_result.unit_test_case_module_path), exist_ok=True)

with open(unit_test_case_result.unit_test_case_module_path, 'w') as fw:
    fw.write(unit_test_case_result.test_case_content)

In [34]:
show_source_code(unit_test_case_result.unit_test_case_module_path)


```python
import unittest
from utils import my_math


class TestMyMath(unittest.TestCase):
    def test_add_positive(self):
        self.assertEqual(my_math.add(1, 1), 2)

    def test_add_negative(self):
        self.assertEqual(my_math.add(-1, -1), -2)

    def test_add_zero(self):
        self.assertEqual(my_math.add(0, 0), 0)

    def test_add_positive_negative(self):
        self.assertEqual(my_math.add(1, -1), 0)
```

#### <b>Execute the generated unit test case file</b>

In [35]:
import unittest
import os
import sys
import importlib
from importlib import reload

# Add the project root to the Python path to enable imports like 'utils.my_math'
# Assuming run_tests.py is at the root and tests/unit/utils/test_my_math.py is the test file
project_root = unit_test_case_result.project_root_path
if project_root not in sys.path:
    sys.path.insert(0, project_root)

class MyCustomTestResult(unittest.TextTestResult):
    def __init__(self, stream, descriptions, verbosity):
        super().__init__(stream, descriptions, verbosity)
        # This is where 'all_test_results' is defined!
        self.all_test_results = []

    def addSuccess(self, test):
        super().addSuccess(test)
        self.all_test_results.append(f"{test} ... ok")

    def addFailure(self, test, err):
        super().addFailure(test, err)
        self.all_test_results.append(f"{test} ... FAILED")

    def addError(self, test, err):
        super().addError(test, err)
        self.all_test_results.append(f"{test} ... ERROR")

    def addSkip(self, test, reason):
        super().addSkip(test, reason)
        self.all_test_results.append(f"{test} ... SKIPPED")


class MyCustomTestRunner(unittest.TextTestRunner):
    def _makeResult(self):
        # This method tells the runner to use YOUR custom result class
        return MyCustomTestResult(self.stream, self.descriptions, self.verbosity)
        

class UnitTestRunner:
    def __init__(self, unit_test_case: UnitTestCases):
        self.unit_test_case = unit_test_case
        self._test_result = None

    def _generate_output(self, result) -> str:
        output_messages = []
        output_messages.append("--- Custom Test Output ---")
        for outcome_string in result.all_test_results:
            output_messages.append(outcome_string)

        output_messages.append('=' * 10)
        if result.wasSuccessful():
            output_messages.append("Overall Result: OK (All tests passed) ✅")
        else:
            output_messages.append("Overall Result: FAILED ❌")
        
        if result.failures:
            output_messages.append(f"Failures ({len(result.failures)}):")
            for test, traceback_str in result.failures:
                output_messages.append(f"  Test: {test}")
                output_messages.append(f"  Traceback:\n{traceback_str}\n")
        if result.errors:
            output_messages.append(f"Errors ({len(result.errors)}):")
            for test, traceback_str in result.errors:
                output_messages.append(f"  Test: {test}")
                output_messages.append(f"  Traceback:\n{traceback_str}\n")

        if result.skipped:
            output_messages.append(f"Skipped ({len(result.skipped)}):")
            for test, reason in result.skipped:
                output_messages.append(f"  Test: {test}, Reason: {reason}\n")
        if result.expectedFailures:
            output_messages.append(f"Expected Failures ({len(result.expectedFailures)}):")
            for test, traceback_str in result.expectedFailures:
                output_messages.append(f"  Test: {test}")
                output_messages.append(f"  Traceback:\n{traceback_str}\n")
        if result.unexpectedSuccesses:
            output_messages.append(f"Unexpected Successes ({len(result.unexpectedSuccesses)}):")
            for test in result.unexpectedSuccesses:
                output_messages.append(f"  Test: {test}\n")

        return '\n'.join(output_messages)
        
    def run(self) -> str:
        project_root = self.unit_test_case.project_root_path
        if project_root not in sys.path:
            sys.path.insert(0, project_root)
        
        # Output unit test cases into testing module.
        self.unit_test_case.output()

        # Ensure the test module is reloaded each time.
        #module_name = self.unit_test_case.test_module_name.split(".")[0]
    
        # Find the fully qualified module path
        #fq_module_name = f"{self.unit_test_case.unit_test_case_module_dir_path}.{module_name}".replace('/', '.')
    
        # Remove from sys.modules if already loaded
        fq_module_name = self.unit_test_case.unit_test_case_module_package
        print(f'Removing module "{fq_module_name}"')
        if fq_module_name in sys.modules:
            del sys.modules[fq_module_name]

        test_module = importlib.import_module(fq_module_name)
        reload(test_module)

        # Optionally clear all test modules from cache
        for name in list(sys.modules.keys()):
            if name.startswith(self.unit_test_case.module_package) and "test_" in name:
                del sys.modules[name]

        # Discover tests from the 'tests' directory
        # start_dir: The directory to start searching for tests (e.g., 'tests')
        # pattern: Only look for files matching this pattern (e.g., 'test_*.py')
        test_loader = unittest.TestLoader()
        print(f'Start dir: {self.unit_test_case.unit_test_case_module_dir_path}')
        print(f'Test module name: {self.unit_test_case.test_module_name.split(".")[0]}')

        #test_suite = test_loader.discover(
        #    start_dir=self.unit_test_case.unit_test_case_module_dir_path,
        #    pattern=self.unit_test_case.test_module_name)
        test_suite = test_loader.loadTestsFromModule(test_module)
        
        # Run the tests
        runner = MyCustomTestRunner(verbosity=0) # verbosity=2 shows more details
        self._test_result = runner.run(test_suite)
        return self._generate_output(self._test_result)

In [36]:
sys.path

['/home/john/Gitrepos/ml_articles/others/Langfun_with_unittest',
 '/usr/lib/python312.zip',
 '/usr/lib/python3.12',
 '/usr/lib/python3.12/lib-dynload',
 '',
 '/home/john/Gitrepos/ml_articles/env/lib/python3.12/site-packages']

In [64]:
test_runner = UnitTestRunner(unit_test_case_result)

In [65]:
print(test_runner.run())

----------------------------------------------------------------------
Ran 4 tests in 0.000s

OK


Removing module "tests.unit.utils.test_my_math"
Start dir: tests/unit/utils
Test module name: test_my_math
--- Custom Test Output ---
test_negative_integers (tests.unit.utils.test_my_math.TestAdd.test_negative_integers) ... ok
test_positive_and_negative (tests.unit.utils.test_my_math.TestAdd.test_positive_and_negative) ... ok
test_positive_integers (tests.unit.utils.test_my_math.TestAdd.test_positive_integers) ... ok
test_zero_sum (tests.unit.utils.test_my_math.TestAdd.test_zero_sum) ... ok
Overall Result: OK (All tests passed) ✅


### <b><font color='darkgreen'>More</font></b>

In [39]:
def create_test_module(
    module_package: str,
    module_name: str,
    project_root_path=None):
    """Creates symbolic object to hold the target module for testing."""
    project_root_path = project_root_path or os.getcwd()
    module_content = open(
        Path(project_root_path) / module_package.replace('.', '/') / f'{module_name}.py', 'r').read()

    return TestModule(
            project_root_path=project_root_path,
            module_package = module_package,
            module_name=module_name,
            module_content=module_content)

def gen_unit_test_cases(test_module) -> UnitTestCases:
    return lf.query(
        prompt='Please create unit test cases for {{module}}',
        schema=UnitTestCases,
        lm=model,
        module=test_module)

In [40]:
str_helper_module = create_test_module('utils.john', 'str_helper')

In [66]:
str_helper_module

TestModule(project_root_path='/home/john/Gitrepos/ml_articles/others/Langfun_with_unittest', module_package='utils.john', module_name='str_helper', module_content='def to_upper(input_str: str) -> str:\n  """Turns input string into upper case."""\n  return input_str.upper()\n')

In [67]:
str_helper_unit_test_case_info = gen_unit_test_cases(str_helper_module)

In [68]:
str_helper_unit_test_case_info

UnitTestCases(project_root_path='/home/john/Gitrepos/ml_articles/others/Langfun_with_unittest', module_package='utils.john', module_name='str_helper', test_case_content='import unittest\nfrom utils.john.str_helper import to_upper\n\n\nclass TestToUpper(unittest.TestCase):\n    def test_to_upper_empty(self):\n        self.assertEqual(to_upper(""), "")\n\n    def test_to_upper_single_char(self):\n        self.assertEqual(to_upper("a"), "A")\n\n    def test_to_upper_multiple_chars(self):\n        self.assertEqual(to_upper("abc"), "ABC")\n\n    def test_to_upper_mixed_case(self):\n        self.assertEqual(to_upper("aBc"), "ABC")\n\n    def test_to_upper_with_space(self):\n        self.assertEqual(to_upper("a b"), "A B")\n\n    def test_to_upper_with_special_chars(self):\n        self.assertEqual(to_upper("!@#$"), "!@#$")\n\n    def test_to_upper_already_upper(self):\n        self.assertEqual(to_upper("ABC"), "ABC")\n\n    def test_to_upper_non_ascii(self):\n        self.assertEqual(to_upper("你好"), "你好") # Non-ASCII characters should remain unchanged\n\n')

In [69]:
runner = UnitTestRunner(str_helper_unit_test_case_info)

In [70]:
output = runner.run()
print(output)

----------------------------------------------------------------------
Ran 8 tests in 0.001s

OK


Removing module "tests.unit.utils.john.test_str_helper"
Start dir: tests/unit/utils/john
Test module name: test_str_helper
--- Custom Test Output ---
test_to_upper_already_upper (tests.unit.utils.john.test_str_helper.TestToUpper.test_to_upper_already_upper) ... ok
test_to_upper_empty (tests.unit.utils.john.test_str_helper.TestToUpper.test_to_upper_empty) ... ok
test_to_upper_mixed_case (tests.unit.utils.john.test_str_helper.TestToUpper.test_to_upper_mixed_case) ... ok
test_to_upper_multiple_chars (tests.unit.utils.john.test_str_helper.TestToUpper.test_to_upper_multiple_chars) ... ok
test_to_upper_non_ascii (tests.unit.utils.john.test_str_helper.TestToUpper.test_to_upper_non_ascii) ... ok
test_to_upper_single_char (tests.unit.utils.john.test_str_helper.TestToUpper.test_to_upper_single_char) ... ok
test_to_upper_with_space (tests.unit.utils.john.test_str_helper.TestToUpper.test_to_upper_with_space) ... ok
test_to_upper_with_special_chars (tests.unit.utils.john.test_str_helper.TestToUpper

### <b><font color='darkgreen'>How to fix generated failed test cases</font></b>
Let's prepare the symbolic unit test case info object with failed test cases:

In [46]:
str_helper_unit_test_case_info

UnitTestCases(project_root_path='/home/john/Gitrepos/ml_articles/others/Langfun_with_unittest', module_package='utils.john', module_name='str_helper', test_case_content="import unittest\nfrom utils.john.str_helper import to_upper\n\n\nclass TestToUpper(unittest.TestCase):\n    def test_to_upper_empty(self):\n        self.assertEqual(to_upper(''), '')\n\n    def test_to_upper_single_char(self):\n        self.assertEqual(to_upper('a'), 'A')\n\n    def test_to_upper_multiple_chars(self):\n        self.assertEqual(to_upper('hello'), 'HELLO')\n\n    def test_to_upper_mixed_case(self):\n        self.assertEqual(to_upper('hELLo'), 'HELLO')\n\n    def test_to_upper_with_space(self):\n        self.assertEqual(to_upper('hello world'), 'HELLO WORLD')\n\n    def test_to_upper_with_number(self):\n        self.assertEqual(to_upper('123'), '123')\n\n    def test_to_upper_special_character(self):\n        self.assertEqual(to_upper('!@#'), '!@#')\n\n    def test_to_upper_alphanumeric_and_special_character(self):\n        self.assertEqual(to_upper('hELLo123!@#'), 'HELLO123!@#')\n\n    def test_to_upper_unicode(self):\n        self.assertEqual(to_upper('你好'), '你好')\n\n")

In [71]:
def change_test_module_name(new_module_name, orig_unit_test_case_info):
    return UnitTestCases(
        test_case_content=orig_unit_test_case_info.test_case_content,
        project_root_path=str_helper_unit_test_case_info.project_root_path,
        module_package=str_helper_unit_test_case_info.module_package,
        module_name=new_module_name)

In [72]:
with open('failed_str_helper_unit_test_cases.py', 'r') as fo:
    failed_str_helper_unit_test_case_info = UnitTestCases(
        test_case_content=fo.read(),
        project_root_path=str_helper_unit_test_case_info.project_root_path,
        module_package=str_helper_unit_test_case_info.module_package,
        module_name='str_helper_v3')

In [73]:
failed_str_helper_unit_test_case_info

UnitTestCases(project_root_path='/home/john/Gitrepos/ml_articles/others/Langfun_with_unittest', module_package='utils.john', module_name='str_helper_v3', test_case_content='import unittest\nfrom utils.john.str_helper import to_upper\n\n\nclass TestToUpper(unittest.TestCase):\n  def test_to_upper_empty(self):\n    self.assertEqual(to_upper(""), "")\n\n  def test_to_upper_single_char(self):\n    self.assertEqual(to_upper("a"), "A")\n\n  def test_to_upper_mixed_case(self):\n    self.assertEqual(to_upper("aBcDe"), "ABCDE")\n\n  def test_to_upper_with_space(self):\n    self.assertEqual(to_upper("abc def"), "ABC DEF")\n\n  def test_to_upper_with_number(self):\n    self.assertEqual(to_upper("abc123def"), "ABC123DEF")\n\n  def test_to_upper_with_special_chars(self):\n    self.assertEqual(to_upper("!@#%^"), "!@#%^")\n\n  def test_to_upper_with_unicode(self):\n    self.assertEqual(to_upper("你好世界"), "你好世界")\n\n  def test_to_upper_None_input(self):\n    with self.assertRaises(AttributeError):\n      to_upper(None)\n\n  def test_to_upper_int_no_upper(self):\n    with self.assertRaises(ValueError):\n      to_upper(123)\n')

In [74]:
runner2 = UnitTestRunner(failed_str_helper_unit_test_case_info)

In [75]:
output = runner2.run()
print(output)

ERROR: test_to_upper_int_no_upper (tests.unit.utils.john.test_str_helper_v3.TestToUpper.test_to_upper_int_no_upper)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/home/john/Gitrepos/ml_articles/others/Langfun_with_unittest/tests/unit/utils/john/test_str_helper_v3.py", line 33, in test_to_upper_int_no_upper
    to_upper(123)
  File "/home/john/Gitrepos/ml_articles/others/Langfun_with_unittest/utils/john/str_helper.py", line 3, in to_upper
    return input_str.upper()
           ^^^^^^^^^^^^^^^
AttributeError: 'int' object has no attribute 'upper'

----------------------------------------------------------------------
Ran 9 tests in 0.002s

FAILED (errors=1)


Removing module "tests.unit.utils.john.test_str_helper_v3"
Start dir: tests/unit/utils/john
Test module name: test_str_helper_v3
--- Custom Test Output ---
test_to_upper_None_input (tests.unit.utils.john.test_str_helper_v3.TestToUpper.test_to_upper_None_input) ... ok
test_to_upper_empty (tests.unit.utils.john.test_str_helper_v3.TestToUpper.test_to_upper_empty) ... ok
test_to_upper_int_no_upper (tests.unit.utils.john.test_str_helper_v3.TestToUpper.test_to_upper_int_no_upper) ... ERROR
test_to_upper_mixed_case (tests.unit.utils.john.test_str_helper_v3.TestToUpper.test_to_upper_mixed_case) ... ok
test_to_upper_single_char (tests.unit.utils.john.test_str_helper_v3.TestToUpper.test_to_upper_single_char) ... ok
test_to_upper_with_number (tests.unit.utils.john.test_str_helper_v3.TestToUpper.test_to_upper_with_number) ... ok
test_to_upper_with_space (tests.unit.utils.john.test_str_helper_v3.TestToUpper.test_to_upper_with_space) ... ok
test_to_upper_with_special_chars (tests.unit.utils.john.tes

In [76]:
fix_str = lf.query(
    prompt='How to fix unit test cases  {{failed}} of {{module}} with outut as:' + output,
    #schema=UnitTestCases,
    module=str_helper_module,
    failed=failed_str_helper_unit_test_case_info,
    lm=model)

In [77]:
print(fix_str)

The error message is clear: `AttributeError: 'int' object has no attribute 'upper'`.  Your `to_upper` function tries to call the `.upper()` method on the input, but integers don't have this method (only strings do).

You have two main options to fix this:

**1. Type Hinting Enforcement (Recommended):**

Use type hinting and a tool like `mypy` to catch this error during development *before* you run your tests.  This is the best approach for preventing such errors in the first place.

```python
def to_upper(input_str: str) -> str:  # Type hint clearly states input must be a string
  """Turns input string into upper case."""
  return input_str.upper()

```

Then, in your terminal, run `mypy utils/john/str_helper.py`.  `mypy` will flag the error when you try to call `to_upper` with an integer.

**2. Explicit Type Checking (Less Recommended):**

Add a type check within your `to_upper` function to raise a more informative error or handle the integer input gracefully.  While this works, it's 

In [81]:
str_helper_unit_test_cases_v2 = lf.query(
    prompt='According to the message: {{fix}}, please fix {{failed}} by replacing it with correct exception `AttributeError`.',
    schema=UnitTestCases,
    fix=fix_str,
    failed=failed_str_helper_unit_test_case_info,
    lm=model)

In [82]:
str_helper_unit_test_cases_v2

UnitTestCases(project_root_path='/home/john/Gitrepos/ml_articles/others/Langfun_with_unittest', module_package='utils.john', module_name='str_helper_v3', test_case_content='import unittest\nfrom utils.john.str_helper import to_upper\n\n\nclass TestToUpper(unittest.TestCase):\n  def test_to_upper_empty(self):\n    self.assertEqual(to_upper(""), "")\n\n  def test_to_upper_single_char(self):\n    self.assertEqual(to_upper("a"), "A")\n\n  def test_to_upper_mixed_case(self):\n    self.assertEqual(to_upper("aBcDe"), "ABCDE")\n\n  def test_to_upper_with_space(self):\n    self.assertEqual(to_upper("abc def"), "ABC DEF")\n\n  def test_to_upper_with_number(self):\n    self.assertEqual(to_upper("abc123def"), "ABC123DEF")\n\n  def test_to_upper_with_special_chars(self):\n    self.assertEqual(to_upper("!@#%^"), "!@#%^")\n\n  def test_to_upper_with_unicode(self):\n    self.assertEqual(to_upper("你好世界"), "你好世界")\n\n  def test_to_upper_None_input(self):\n    with self.assertRaises(AttributeError):\n      to_upper(None)\n\n  def test_to_upper_int_no_upper(self):\n    with self.assertRaises(AttributeError):\n      to_upper(123)\n')

In [83]:
# change_test_module_name('str_helper_v6', str_helper_unit_test_cases_v2)
runner = UnitTestRunner(str_helper_unit_test_cases_v2)
output = runner.run()

----------------------------------------------------------------------
Ran 9 tests in 0.001s

OK


Removing module "tests.unit.utils.john.test_str_helper_v3"
Start dir: tests/unit/utils/john
Test module name: test_str_helper_v3


In [98]:
print(output)

--- Custom Test Output ---
test_to_upper_None_input (test_str_helper_v7.TestToUpper.test_to_upper_None_input) ... ok
test_to_upper_empty (test_str_helper_v7.TestToUpper.test_to_upper_empty) ... ok
test_to_upper_int_no_upper (test_str_helper_v7.TestToUpper.test_to_upper_int_no_upper) ... ok
test_to_upper_mixed_case (test_str_helper_v7.TestToUpper.test_to_upper_mixed_case) ... ok
test_to_upper_single_char (test_str_helper_v7.TestToUpper.test_to_upper_single_char) ... ok
test_to_upper_with_number (test_str_helper_v7.TestToUpper.test_to_upper_with_number) ... ok
test_to_upper_with_space (test_str_helper_v7.TestToUpper.test_to_upper_with_space) ... ok
test_to_upper_with_special_chars (test_str_helper_v7.TestToUpper.test_to_upper_with_special_chars) ... ok
test_to_upper_with_unicode (test_str_helper_v7.TestToUpper.test_to_upper_with_unicode) ... ok
Overall Result: OK (All tests passed) ✅


## <b><font color='darkblue'>Supplement</font></b>
* [Colab - Langfun 101: Getting Started with Langfun](https://colab.research.google.com/github/google/langfun/blob/main/docs/notebooks/langfun101.ipynb)